#### 1. 환경 설정

##### 1.1. 라이브러리 설치

In [ ]:
"""
!pip install -U transformers
!pip install -U pytorch_lightning
!pip install -U datasets
!pip install -U rouge
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U trl
!pip install -U accelerate
!pip install -U evaluate
"""

##### 1.2. 모듈 불러오기

In [3]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl

from sklearn.model_selection import train_test_split
from rouge import Rouge

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

os.environ["TOKENIZERS_PARALLELISM"] = "true"

#### 2. 데이터셋 준비

In [4]:
data_path = '../data/'

train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
dev_df = pd.read_csv(os.path.join(data_path, 'dev.csv'))

# 8 대 2 비율로 train, validation dataset 생성
train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state=2024)

In [ ]:
train_df.info()

#### 3. 프롬프트 생성

##### 3.1 모델 별 Special Token 살펴보기

In [6]:
def print_special_tokens(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    print(f"bos token: {tokenizer.bos_token}")
    print(f"eos token: {tokenizer.eos_token}")
    print(f"pad token: {tokenizer.pad_token}")

In [ ]:
# 인코더 모델 : RoBERTa
from transformers import AutoModel, AutoTokenizer

MODEL_NAME = 'kykim/bert-kor-base'

print_special_tokens(MODEL_NAME)

In [ ]:
# 디코더 모델 : LLaMA
from transformers import AutoModelForCausalLM #, AutoTokenizer

# MODEL_NAME = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
MODEL_NAME = 'unsloth/Llama-3.2-1B'

print_special_tokens(MODEL_NAME)

In [ ]:
# 인코더 - 디코더 모델 : T5
from transformers import AutoModelForSeq2SeqLM #, AutoTokenizer

MODEL_NAME ='eenzeenee/t5-base-korean-summarization'

print_special_tokens(MODEL_NAME)

##### 3.2 모델 별 학습용 프롬프트 생성 함수

In [ ]:
# RoBERTa
# 입력 텍스트 중심의 task 정의, <s> 토큰으로 시작과 종료를 알린다.

'''
def train_prompt_roberta(row):

    topic = row['topic'] if 'topic' in row else ''

    prompt = f"""<s> 다음의 대화를 요약하세요. 아래의 지침을 따르세요:

    1. 대화 길이의 20% 이내로 요약
    2. 대화 내에서 중요하게 명명된 개체의 이름을 그대로 사용
    3. 은어나 약어 없이 공식적으로 사용하는 언어로 작성

    주제: {topic}
    대화: {row['dialogue']}
    요약: {row['summary']} </s>"""

    return prompt
'''

In [ ]:
# LLaMA
# 시스템 메세지 및 어시스턴트 응답을 활용하여 명확히 구분한다.

'''
def train_prompt_llama(row):
    topic = row['topic'] if 'topic' in row else ''

    prompt = f"""<s> <|im_start|>system
    주제를 참고해 다음 대화를 요약하세요. 아래의 지침을 따르세요:

    1. 대화 길이의 20% 이내로 요약
    2. 대화 내에서 중요하게 명명된 개체의 이름을 그대로 사용
    3. 은어나 약어 없이 공식적으로 사용하는 언어로 작성

    주제: {topic}
    대화: {row['dialogue']}
    <|im_end|>
    <|im_start|>assistant
    요약: {row['summary']}
    <|im_end|> </s>"""

    return prompt
'''


In [12]:
# T5
# task 이름을 명시적으로 포함하고, Seq2Seq 형식으로 작성한다.

def train_prompt_t5(row):
    topic = row['topic'] if 'topic' in row else ''

    prompt = f"""summarize dialogue:
    topic: {topic}
    dialogue: {row['dialogue']}

    summary: {row['summary']}"""

    return prompt


##### 3.3 프롬프트 적용

In [ ]:
train_df['prompt'] = train_df.apply(train_prompt_t5, axis=1)
valid_df['prompt'] = valid_df.apply(train_prompt_t5, axis=1)

train_df.head(3)

#### 4. 모델 경량화

##### 4.1. LoRA Config 작성

In [17]:
from peft import LoraConfig, TaskType, PeftModel, get_peft_model

lora_config = LoraConfig(
    r=4,

    # Attention 모듈 : q_proj, k_proj, v_proj, o_proj
    # MLP 모듈 : gate_proj, down_proj, up_proj

    # target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    target_modules=["q", "v", "wi", "wo"],

    # task_type=TaskType.CAUSAL_LM,

    # T5
    task_type=TaskType.SEQ_2_SEQ_LM,
    lora_alpha=32,
    lora_dropout=0.05
)

##### 4.2. Quantization Config 작성

In [18]:
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

##### 4.3. 적용

In [ ]:
# AutoModelForCausalLM

# T5
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    device_map={"": "cuda"},
    quantization_config=quant_config,
    torch_dtype=torch.float16,
)

In [21]:
# LoRA 적용
model = get_peft_model(model, lora_config)

#### 5. 모델 학습

##### 5.1. 평가 지표

In [22]:
# 3.0 이상의 버전에서는 load_metric()을 지원하지 않는다. evalute library 사용
import evaluate
from rouge_score import rouge_scorer, scoring
import numpy as np
import gc

# ROUGE Metric
rouge = evaluate.load('rouge')

def rouge_metric(pred, tokenizer):

    labels_ids = pred.labels_ids
    pred_ids = pred.predictions

    # Decoding
    decoded_preds = []
    decoded_labels = []

    for pred_id, label_id in zip(pred_ids, labels_ids):

        decoded_preds.append(tokenizer.decode(pred_id, skip_special_tokens=True))
        decoded_labels.append(tokenizer.decode(label_id, skip_special_tokens=True))

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_aggregator=True,
    )

    del decoded_preds, decoded_labels, labels_ids, pred_ids
    gc.collect()

    return np.mean(result['rouge1'] + result['rouge2'] + result['rougeL'])

##### 5.2. Special Token 추가

In [ ]:
special_tokens = ['#PassportNumber#', '#CardNumber#', '#Person3#', '#DateOfBirth#', '#Address#', '#CarNumber#', '#Email#',
                  '#Person2#', '#Person6#', '#Person1#', '#Person#', '#Person7#', '#Person5#', '#PhoneNumber#', '#Person4#', '#SSN#']

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

# 중요
model.resize_token_embeddings(len(tokenizer))

##### 5.3. 학습 진행

In [ ]:
from datasets import Dataset
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# 학습 데이터 준비
train_data = Dataset.from_dict({'prompt': train_df['prompt']})
valid_data = Dataset.from_dict({'prompt': valid_df['prompt']})

def formatting_func(dataframe):
    return dataframe['prompt']

# Supervised Fine-Tuning Trainer
trainer = SFTTrainer(
    model=model,

    train_dataset=train_data,
    eval_dataset=valid_data,

    args=TrainingArguments(
        seed=2024,
        output_dir='./results',
        logging_dir='./logs',

        num_train_epochs=3,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        eval_accumulation_steps=4,
        dataloader_num_workers=2,

        fp16=True,

        optim='adamw_torch_fused',
        warmup_steps=50,
        learning_rate=2e-4,
        lr_scheduler_type='cosine',
        max_grad_norm=0.3,
        weight_decay=0.001,

        logging_steps=200,
        save_steps=1000,
        load_best_model_at_end=True,
        evaluation_strategy='steps',
        metric_for_best_model='eval_loss',

    ),
    peft_config=lora_config,

    compute_metrics=rouge_metric,
    tokenizer=tokenizer,

    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],

    formatting_func=formatting_func,
)

In [ ]:
# 훈련 시작
trainer.train()

In [ ]:
# Save Best Model
TUNED_MODEL = "best_model_t5" # "best_model_roberta" # "best_model_llama"

trainer.model.save_pretrained(TUNED_MODEL)

model = PeftModel.from_pretrained(model, TUNED_MODEL, device_map={"": "cuda"}, torch_dtype=torch.float16)
model.save_pretrained('fine-tuned-t5-ko')

In [ ]:
# 메모리 정리
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

del model
del trainer

##### 6. 모델 불러오기

In [ ]:
FINE_TUNED_MODEL = 'fine-tuned-t5-ko'

#AutoModelForCausalLM
model = AutoModelForSeq2SeqLM.from_pretrained(
    FINE_TUNED_MODEL,
    device_map={"": "cuda"},
    torch_dtype=torch.float16,
    device_map='auto'
)

special_tokens = ['#PassportNumber#', '#CardNumber#', '#Person3#', '#DateOfBirth#', '#Address#', '#CarNumber#', '#Email#',
                  '#Person2#', '#Person6#', '#Person1#', '#Person#', '#Person7#', '#Person5#', '#PhoneNumber#', '#Person4#', '#SSN#']

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

model.resize_token_embeddings(len(tokenizer))

#### 7. 모델 평가하기

##### 7.1 평가용 프롬프트 작성

In [ ]:
# RoBERTa

'''
def test_prompt_roberta(row):

    topic = row['topic'] if 'topic' in row else ''

    prompt = f"""<s> 다음의 대화를 요약하세요. 아래의 지침을 따르세요:

    1. 대화 길이의 20% 이내로 요약
    2. 대화 내에서 중요하게 명명된 개체의 이름을 그대로 사용
    3. 은어나 약어 없이 공식적으로 사용하는 언어로 작성

    주제: {topic}
    대화: {row['dialogue']}
    요약: </s>"""

    return prompt
'''

In [4]:
# LLaMA

'''
def test_prompt_llama(row):
    topic = row['topic'] if 'topic' in row else ''

    prompt = f"""<s> <|im_start|>system
    주제를 참고해 다음 대화를 요약하세요. 아래의 지침을 따르세요:

    1. 대화 길이의 20% 이내로 요약
    2. 대화 내에서 중요하게 명명된 개체의 이름을 그대로 사용
    3. 은어나 약어 없이 공식적으로 사용하는 언어로 작성

    주제: {topic}
    대화: {row['dialogue']}
    <|im_end|>
    <|im_start|>assistant
    요약:
    <|im_end|> </s>"""

    return prompt
'''


In [ ]:
# T5

def test_prompt_t5(row):
    topic = row['topic'] if 'topic' in row else ''

    prompt = f"""summarize dialogue:
    topic: {topic}
    dialogue: {row['dialogue']}

    summary: """

    return prompt


In [5]:
test_df = pd.read_csv(os.path.join(data_path, 'test.csv'))

In [ ]:
test_df['prompt'] = test_df.apply(test_prompt_t5, axis=1)
test_df.head(3)

##### 7.2 요약문 생성 함수

In [22]:
from tqdm import tqdm
from transformers import TextStreamer

def generate_summary(prompt, model, tokenizer):

    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    output = model.generate(
        **inputs,
        streamer=TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=False),
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=50,
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)

    exclude_words = ["assistant", "system", "imagine", "imports", "generator","ai_generated_output", "sym_daily","AI_Explanation","label","endorsements","parameters","original","model","implicit_template"]
    for word in exclude_words:
        if word in output:
            output = output.split(word)[0].strip()

    return output

In [ ]:
test_df['summary'] = ''

for i, row in tqdm(test_df.iterrows(), total=len(test_df), unit='example', leave=True):

    prompt = row['prompt']
    summary = generate_summary(prompt, model, tokenizer)

    if i % 20 == 0:
        print(summary)
    test_df.at[i, 'summary'] = summary

In [ ]:
submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
submission['summary'] = test_df['summary']

In [ ]:
submission['summary'].head(3)

In [ ]:
submission.to_csv('./output_t5.csv', index=False) # './output_roberta.csv', './output_llama.csv'